In [1]:
import logging
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from data_preprocessing import DataPreprocessor
from model_classifier import CrossSellingModel
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import logging_config

from utils import load_df_from_zip

logging_config.configure_logging()
LOGGER = logging.getLogger(__name__)

In [2]:
train = load_df_from_zip('data/train.zip')

In [3]:
X = train.drop('Response', axis=1)
y = train['Response']

majority_class_samples = 1415000
df_undersampled_majority = resample(train[y==0], replace=False, n_samples=majority_class_samples, random_state=42)
train = pd.concat([df_undersampled_majority, train[y==1]])

In [4]:
X = train.drop(columns='Response', axis=1)
y = train['Response']

preprocessor = DataPreprocessor(X)
X = preprocessor.preprocess_data()
# X = DataPreprocessor(X).preprocess_data()
LOGGER.info(f"X: {X.shape}")

[2024-07-23 16:31:45,122 : INFO : X: (2830059, 15)]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
LOGGER.info(f"X_train.shape: {X_train.shape}")
LOGGER.info(f"X_test.shape: {X_test.shape}")
LOGGER.info(f"y_train.shape: {y_train.shape}")
LOGGER.info(f"y_test.shape: {y_test.shape}")

[2024-07-23 16:31:45,704 : INFO : X_train.shape: (2264047, 15)]
[2024-07-23 16:31:45,705 : INFO : X_test.shape: (566012, 15)]
[2024-07-23 16:31:45,707 : INFO : y_train.shape: (2264047,)]
[2024-07-23 16:31:45,708 : INFO : y_test.shape: (566012,)]


In [6]:
# # Convert y_train to numpy array
# y_train_array = y_train.to_numpy()
# 
# # Get unique classes
# classes = np.unique(y_train_array)
# 
# # Compute class weights
# class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_array)
# 
# # Create dictionary of class weights
# class_weight_dict = dict(zip(classes, class_weights))

In [7]:
# print(class_weight_dict)
# print(y_train.unique())
# class_weight_dict = {int(k): v for k, v in class_weight_dict.items()}
# print(class_weight_dict)
# print(type(y_train))
# y_train = y_train.reset_index(drop=True)
# y_test = y_test.reset_index(drop=True)
# 
# y_train = y_train.squeeze()
# y_test = y_test.squeeze()
# 
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()

In [ ]:
classifier = CrossSellingModel(X.shape[1])
# classifier.train_model(X_train, y_train, X_test, y_test, class_weights=class_weight_dict, epochs=10, batch_size=64)
classifier.train_model(X_train, y_train, X_test, y_test, class_weights=None, epochs=10, batch_size=128)

Epoch 1/10


In [ ]:
# test = load_df_from_zip('data/test.zip')
# ids = test['id']
# test_pr = preprocessor.preprocessor.transform(test)
# # test = DataPreprocessor(test).preprocess_data(test)

In [ ]:
# import pandas as pd
# 
# predictions = classifier.get_predictions(test_pr, 0.5)
# submission = pd.DataFrame({'id': ids, 'Response': predictions})
# submission.to_csv('submission.csv', index=False)

In [ ]:
# classifier.save_model('cross_selling_model.keras')